In [1]:
import requests
import json
import pandas as pd


In [2]:
url ="http://publicplansdata.org/api/"
#http://publicplansdata.org/api/?q=QVariables&variables=fy,PlanName,PercentReqContPaid,InvestmentReturn_5yr,ActFundedRatio_GASB&filterfystart=2000&filterfyend=2019&format=json

In [3]:
params = {
    "q": "ListDataSets",
     "format": "json"
    
}


In [4]:
response = requests.get(url, params = params)
print(response.url)

http://publicplansdata.org/api/?q=ListDataSets&format=json


In [5]:
data = response.json()
data

[{'status': 'OK',
  'date': '2021-05-27 05:23:11',
  'q': 'listdatasets',
  'params': 'q=ListDataSets&format=json',
  'recordcount': 34},
 {'id': '3',
  'table_name': 'pensiongasbschedules',
  'table_display_name': 'Funding'},
 {'id': '4',
  'table_name': 'pensiongasbassumptions',
  'table_display_name': 'Assumptions and Methods'},
 {'id': '5',
  'table_name': 'pensionincomestatement',
  'table_display_name': 'Income Statement'},
 {'id': '7',
  'table_name': 'pensioninvestmentreturn',
  'table_display_name': 'Investment Returns'},
 {'id': '8',
  'table_name': 'pensionactuarialcosts',
  'table_display_name': 'Actuarial Costs'},
 {'id': '9',
  'table_name': 'pensionprovisions',
  'table_display_name': 'Plan Provisions'},
 {'id': '10',
  'table_name': 'pensionmembership',
  'table_display_name': 'Membership'},
 {'id': '11',
  'table_name': 'pensionreportingdates',
  'table_display_name': 'Plan Reporting'},
 {'id': '12',
  'table_name': 'pensioneegroupbasics',
  'table_display_name': 'Empl

In [6]:
#Make sure to use QDataset, NOT QDataSet.  Makes a big difference
paramsratios = {
    "q": "QDataset",
    "dataset": "pensiongasbschedules",
     "format": "json",
    #"variables": 
     #"v": variables,
}
paramsbasicplandata = {
    "q": "QDataset",
    "dataset": "pensionplanbasics",
     "format": "json",
    #"variables": 
     #"v": ,
}


In [7]:
ratiosresponse = requests.get(url, params = paramsratios)
#print(ratiosresponse.url)
plandataresponse = requests.get(url,params=paramsbasicplandata)

In [8]:
fundedratiosdata = ratiosresponse.json()
plandata = plandataresponse.json()

In [9]:
funded_ratios_full = pd.DataFrame(fundedratiosdata[1:])
plan_data_full = pd.DataFrame(plandata[1:])

In [10]:
funded_ratios_full.columns

Index(['ActAssets_est', 'ActAssets_GASB', 'ActFundedRatio_est',
       'ActFundedRatio_GASB', 'ActFundedRatio_GASB67', 'ActLiabilities_est',
       'ActLiabilities_GASB', 'ActLiabilities_other', 'ADEC', 'EEGroupID',
       'fy', 'NetPensionLiability', 'NetPosition', 'notes_GASBSchedules',
       'payroll', 'PercentReqContPaid', 'ppd_id', 'RequiredContribution',
       'RequiredContribution_est', 'source_GASBSchedules', 'TierID',
       'TotalPensionLiability', 'UAAL_GASB'],
      dtype='object')

In [11]:
funded_ratio_df=funded_ratios_full[['ppd_id','fy','ActFundedRatio_GASB','ActFundedRatio_est']].dropna()
funded_ratio_df.shape

(4032, 4)

In [12]:
plan_data_full.columns

Index(['AdministeringGovt', 'BenefitsWebsite', 'CostSharing', 'CostStructure',
       'DataEndYear', 'DataStartYear', 'EmployeeTypeCovered', 'EmployerType',
       'FiscalYearType', 'GovtName', 'InPFS', 'notes_PlanBasics', 'PlanClosed',
       'PlanFullName', 'PlanInceptionYear', 'PlanName', 'PlanType',
       'PlanYearClosed', 'ppd_directory_listed', 'ppd_id', 'SocSecCovered',
       'SocSecCovered_verbatim', 'source_PlanBasics', 'StateAbbrev',
       'StateName', 'system_id'],
      dtype='object')

In [13]:
plan_data_df=plan_data_full[['ppd_id','PlanName','StateAbbrev','PlanClosed','PlanYearClosed',]]
plan_data_df.head()

,ppd_id,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
0,1,Alabama ERS,AL,0,None
1,2,Alabama Teachers,AL,0,None
2,3,Alaska PERS,AK,1,2006
3,4,Alaska Teachers,AK,1,2006
4,5,Arizona Public Safety,AZ,0,None


In [14]:
plan_test_data_df=pd.merge(funded_ratio_df,plan_data_df,how="inner",on="ppd_id")
plan_test_data_df.shape

(4032, 8)

In [15]:
plan_test_data_df.head(500)

,ppd_id,fy,ActFundedRatio_GASB,ActFundedRatio_est,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
0,1,2001,1.00200,0.000,Alabama ERS,AL,0,None
1,1,2002,0.95400,0.000,Alabama ERS,AL,0,None
2,1,2003,0.91100,0.000,Alabama ERS,AL,0,None
3,1,2004,0.89700,0.000,Alabama ERS,AL,0,None
4,1,2005,0.84000,0.000,Alabama ERS,AL,0,None
...,...,...,...,...,...,...,...,...
495,25,2011,0.76450,0.000,Fairfax County Schools,VA,0,None
496,25,2012,0.75560,0.000,Fairfax County Schools,VA,0,None
497,25,2013,0.75420,0.000,Fairfax County Schools,VA,0,None
498,25,2014,0.76700,0.000,Fairfax County Schools,VA,0,None
